# Using Earth Engine Data

```{contents}
:local:
:depth: 3
```

## Introduction

## Technical requirements

```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
```

```bash
jupyter lab
```

In [171]:
# %pip install pygis

In [172]:
import ee
import geemap

In [173]:
geemap.ee_initialize()

## Earth Engine data types

### Image

#### Loading Earth Engine images

In [174]:
image = ee.Image('USGS/SRTMGL1_003')

In [175]:
image

Name,Description
elevation,Elevation


In [176]:
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'elevation',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [1296001, 417601],
   'crs': 'EPSG:4326',
   'crs_transform': [0.0002777777777777778,
    0,
    -180.0001388888889,
    0,
    -0.0002777777777777778,
    60.00013888888889]}],
 'id': 'USGS/SRTMGL1_003',
 'version': 1641990767055141,
 'properties': {'system:visualization_0_min': '0.0',
  'type_name': 'Image',
  'keywords': ['dem',
   'elevation',
   'geophysical',
   'nasa',
   'srtm',
   'topography',
   'usgs'],
  'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png',
  'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="https://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by NASA JP

In [177]:
# Para acceder a una parte espeífica de los metadatos
image.getInfo()['bands']

KeyboardInterrupt: 

#### Visualizing Earth Engine images

In [ ]:
# Procedimiento estandar para la visualización de cualquier datset
Map = geemap.Map(center=[21.79, 70.87], zoom=3) # Objeto mapa
image = ee.Image('USGS/SRTMGL1_003') # Puntero apunta a 
vis_params = {
    'min': 0,
    'max': 6000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(image, vis_params, 'SRTM')
Map

Map(center=[21.79, 70.87], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

#### Loading Cloud GeoTIFFs

Esta parte se la salta en la explicación. Basicamente es para los casos en los que tiene almacenadas las imagenes en Google Cloud. Para que sepas que también lo puedes cargar desde ahí. Relativamente "poco" frecuente, lo más normal es 

In [ ]:
Map = geemap.Map()
URL = 'https://bit.ly/3aSZ0fH'
image = geemap.load_GeoTIFF(URL)
vis = {
    "min": 3000,
    "max": 13500,
    "bands": ["B3", "B2", "B1"],
}
Map.addLayer(image, vis, 'Cloud GeoTIFF')
Map.centerObject(image)
Map

EEException: Image.loadGeoTIFF: Failed to load the GeoTIFF at 'gs://pdd-stac/disasters/hurricane-harvey/0831/20170831_172754_101c_3B_AnalyticMS.tif#1519335766223629': The overview at index 1 with dimensions (2770, 1307) is not a power-of-two reduction of the image with dimensions (8310, 3919).

In [ ]:
B3 = 'gs://gcp-public-data-landsat/LC08/01/044/034/LC08_L1TP_044034_20131228_20170307_01_T1/LC08_L1TP_044034_20131228_20170307_01_T1_B3.TIF'
B4 = 'gs://gcp-public-data-landsat/LC08/01/044/034/LC08_L1TP_044034_20131228_20170307_01_T1/LC08_L1TP_044034_20131228_20170307_01_T1_B4.TIF'
B5 = 'gs://gcp-public-data-landsat/LC08/01/044/034/LC08_L1TP_044034_20131228_20170307_01_T1/LC08_L1TP_044034_20131228_20170307_01_T1_B5.TIF'

In [ ]:
URLs = [B3, B4, B5]
collection = geemap.load_GeoTIFFs(URLs)
image = collection.toBands().rename(['Green', 'Red', 'NIR']).selfMask()

In [ ]:
Map = geemap.Map()
vis = {'bands': ['NIR', 'Red', 'Green'], 'min': 100, 'max': 12000, 'gamma': 0.8}
Map.addLayer(image, vis, 'Image')
Map.centerObject(image, 8)
Map

### ImageCollection

#### Loading image collections

In [ ]:
# Acedes a toda la colección de imágenes
collection = ee.ImageCollection('COPERNICUS/S2_SR')

In [ ]:
collection.first()

In [ ]:
collection.limit(5) # Selecciona las 5 primeras

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


#### Visualizing image collections

In [ ]:
ap = geemap.Map()
collection = ee.ImageCollection('COPERNICUS/S2_SR')
image = collection.median() # Usa la media para estimar la 

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B8', 'B4', 'B3'],
}

Map.setCenter(83.277, 17.7009, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(bottom=2051.0, center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

#### Filtering image collections

In [ ]:
Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-01-01', '2022-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) # Aplica un filtro de nubes, inferior 5%
)
image = collection.median()

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B8', 'B4', 'B3'],
}

Map.setCenter(83.277, 17.7009, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### Geometry

#### Geometry types

#### Creating Geometry objects

In [ ]:
Map = geemap.Map()

point = ee.Geometry.Point([1.5, 1.5])

lineString = ee.Geometry.LineString([[-35, -10], [35, -10], [35, 10], [-35, 10]])

linearRing = ee.Geometry.LinearRing(
    [[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]
)

rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20])

polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

Map.addLayer(point, {}, 'Point')
Map.addLayer(lineString, {}, 'LineString')
Map.addLayer(linearRing, {}, 'LinearRing')
Map.addLayer(rectangle, {}, 'Rectangle')
Map.addLayer(polygon, {}, 'Polygon')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
Map = geemap.Map()

point = ee.Geometry.Point([1.5, 1.5])

lineString = ee.Geometry.LineString(
    [[-35, -10], [35, -10], [35, 10], [-35, 10]], None, False
)

linearRing = ee.Geometry.LinearRing(
    [[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]], None, False
)

rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20], None, False)

polygon = ee.Geometry.Polygon(
    [[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]], None, False
)

Map.addLayer(point, {}, 'Point')
Map.addLayer(lineString, {}, 'LineString')
Map.addLayer(linearRing, {}, 'LinearRing')
Map.addLayer(rectangle, {}, 'Rectangle')
Map.addLayer(polygon, {}, 'Polygon')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

#### Ejercicio sobre la marcha: ¿cuantas imágenes de Sentinel cubren el Campo Grande entre 2022 y 2023?


In [ ]:
Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2022-01-01', '2023-01-01'))

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B8', 'B4', 'B3'],
}

Map.setCenter(715.277347, 41.642449, 16)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[41.642449, 715.277347], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

Objetivo 1: ¿Que coordenadas se encuentra el campo grande? 
- Op1. Con GEEmap, al tratarse de una GUI, podemos identificar el punto con la herrramienta de abajo a la izq (poco preciso)
- Op2. Map.center -> nos dice la posición del centro en el visor del mapa. 
- Op3 Map.user_roi -> cuando hemos puesto previamente con el puntero el putno que nos interesa.


In [ ]:
Map.center

[41.64580299560587, 715.2696776390077]

In [ ]:
Map.user_roi # Con esto puedes obtener las coordenadas de la figura que dibujaste (ej. rectángulo)

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Point",
    "arguments": {
      "coordinates": {
        "constantValue": [
          715.269552,
          41.645848
        ]
      }
    }
  }
})

In [ ]:
# A point geometry for the Campo Grande
geom = ee.Geometry.Point(715.2696776390077, 41.64580299560587)

Ahora que ya conocemos el punto, filtramos por geom en la colección -> buscamos que intersecte

In [ ]:
Map = geemap.Map()
collection_campo = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2022-05-01', '2023-07-01')
    .filterBounds(geom)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)))
collection_campo

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [ ]:
print(f"Número de imágenes encontradas:") 
collection_campo.size()

Número de imágenes encontradas:


In [ ]:
image = collection_campo.first()

In [ ]:
vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

In [ ]:
Map.addLayer(image, vis,'First image')
Map.setCenter(715.277347, 41.642449, 15)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(bottom=1562840.0, center=[41.642449, 715.277347], controls=(WidgetControl(options=['position', 'transparen…

#### Using drawing tools

In [ ]:
if Map.user_roi is not None:
    print(Map.user_roi.getInfo())

In [ ]:
if Map.user_roi is not None:
    print(Map.user_roi)

### Feature

#### Creating Feature objects

In [ ]:
polygon = ee.Geometry.Polygon(
    [[[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]], None, False
)
polyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart'})

In [ ]:
polyFeature

In [ ]:
Map = geemap.Map()
Map.addLayer(polyFeature, {}, 'feature')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
props = {'foo': ee.Number(8).add(88), 'bar': 'hello'}
nowhereFeature = ee.Feature(None, props)
nowhereFeature

#### Setting Feature properties

In [ ]:
feature = (
    ee.Feature(ee.Geometry.Point([-122.22599, 37.17605]))
    .set('genus', 'Sequoia')
    .set('species', 'sempervirens')
)
newDict = {'genus': 'Brachyramphus', 'presence': 1, 'species': 'marmoratus'}
feature = feature.set(newDict)
feature

#### Getting Feature properties

In [ ]:
prop = feature.get('species')
prop

In [ ]:
props = feature.toDictionary()
props

### FeatureCollection

#### Loading feature collections

In [ ]:
# Cargamos un FeatureCollection al igual que haciamos con el Imagen Collecion
# Para seleccionar una linea específica utilizamos el inspeccionar del mapa interactivo.
Map = geemap.Map()
fc = ee.FeatureCollection('TIGER/2016/Roads') 
Map.setCenter(-73.9596, 40.7688, 12)
Map.addLayer(fc, {}, 'Census roads')
Map

Map(center=[40.7688, -73.9596], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [ ]:
fc.limit(3)

Name,Type,Description
fullname,STRING,A human-readable road name
linearid,STRING,The primary identifier used to refer to this row in other TIGER products
mtfcc,STRING,"The road priority code, representing, e.g., primary, second, local, etc."
rttyp,STRING,"The route type code,"


#### Creating feature collections

In [ ]:
features = [
    ee.Feature(ee.Geometry.Rectangle(30.01, 59.80, 30.59, 60.15), {'name': 'Voronoi'}),
    ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
    ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'}),
]
fromList = ee.FeatureCollection(features)

#### Filtering feature collections

In [ ]:
Map = geemap.Map()
states = ee.FeatureCollection('TIGER/2018/States')
feat = states.filter(ee.Filter.eq('NAME', 'Texas')) # Filtrar por campo = nombre 
Map.addLayer(feat, {}, 'Texas')
Map.centerObject(feat)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
texas = feat.first()
texas.toDictionary()

In [ ]:
Map = geemap.Map()
states = ee.FeatureCollection('TIGER/2018/States')
fc = states.filter(ee.Filter.inList('NAME', ['California', 'Oregon', 'Washington']))
Map.addLayer(fc, {}, 'West Coast')
Map.centerObject(fc)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-88.40, 29.88, -77.90, 35.39)

fc = ee.FeatureCollection('TIGER/2018/States').filterBounds(region)
Map.addLayer(fc, {}, 'Southeastern U.S.')
Map.centerObject(fc)

Ejercicio propio: filtrar provincia de zamora

In [ ]:
Map = geemap.Map()
provincias = ee.FeatureCollection("FAO/GAUL/2015/level2") # Añadimos el dataset que encontramos en el Catálogo
zam = provincias.filter(ee.Filter.eq('ADM2_NAME', 'Zamora')) # Filtrar por campo = nombre 
Map.addLayer(zam, {}, 'Zamora')
Map.centerObject(zam)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Para filtrar con una selección interactiva --> basta con dibujar una figura incluyendo las provincias que quieres seleccionar, y a continuación filtra usando user_roi

In [ ]:
provincias_seleccion = provincias.filterBounds(Map.user_rois) 

¿¿Cuantas provincias empiezan por pa? -> uso de filtro stringsStartsWitch

In [ ]:
pab_provincias = provincias.filter(ee.Filter.stringStartsWith('ADM2_NAME', 'Pa'))

In [ ]:
Map.addLayer(pab_provincias,{},'pa')
Map

Map(bottom=6286.0, center=[43.54854811091288, 0.22813183631228198], controls=(WidgetControl(options=['position…

In [ ]:
pab_provincias.size()

#### Visualizing feature collections

Objetivo: aprender a visualizar cualquier imagen, de cualquier periodo y de cualquier colección. Algo similar a lo que hacemos cuando vamos al portal interactivo de USGS. 

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
image = ee.Image().paint(states, 0, 3)
Map.addLayer(image, {'palette': 'red'}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:
# Aquí la clave es que añadimos la función style --> disponible en documentación en ee.FeatureCollection -> style function 
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")

# Define un diccionario con los parámetros de style
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000080'}

# Usa los dos puntos para desenvolver el diccionario y que lo pueda reconocer
Map.addLayer(states.style(**style), {}, "US States") 
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'color': '000000',
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColorOpacity': 0.66,
}
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
Map.add_styled_vector(
    states, column="NAME", palette=palette, layer_name="Styled vector", **vis_params
)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

### Ejercicio: 

In [ ]:
Map = geemap.Map()
provincias = ee.FeatureCollection("TIGER/2016/Counties") # Añadimos el dataset que encontramos en el Catálogo
style = {'color': 'ffffffff', 'width': 2, 'lineType': 'dotted', 'fillColor': '00008080'} # Define un diccionario
Map.addLayer(provincias.style(**style), {}, 'Zamora')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

#### Styling by attribute

Nos va a permitir una representación cualitativa!!!! -> por atributos

In [ ]:
Map = geemap.Map(center=[28.00142, -81.7424], zoom=13)
Map.add_basemap('HYBRID')

In [ ]:
## Creamos los tipos y los colores asociados
types = [
    "Freshwater Forested/Shrub Wetland",
    "Freshwater Emergent Wetland",
    "Freshwater Pond",
    "Estuarine and Marine Wetland",
    "Riverine",
    "Lake",
    "Estuarine and Marine Deepwater",
    "Other",
]

colors = [
    "#008837",
    "#7FC31C",
    "#688CC0",
    "#66C2A5",
    "#0190BF",
    "#13007C",
    "#007C88",
    "#B28653",
]

# Lo que hace con este bucle es añadir A8 al final de cada código de color
# Esto se hace para añadirle la opacidad. Por defecto los RGB solo tienen 6 valores, de ahí que necesite añadir 2+
fillColor = [c + "A8" for c in colors]

In [ ]:
fillColor

['#008837A8',
 '#7FC31CA8',
 '#688CC0A8',
 '#66C2A5A8',
 '#0190BFA8',
 '#13007CA8',
 '#007C88A8',
 '#B28653A8']

In [ ]:
fc = ee.FeatureCollection("projects/sat-io/open-datasets/NWI/wetlands/FL_Wetlands")
styled_fc = geemap.ee_vector_style(
    fc, column='WETLAND_TY', labels=types, fillColor=fillColor, color='00000000'
)

In [ ]:
Map.addLayer(styled_fc, {}, 'NWI')
Map.add_legend(title='Wetland Type', labels=types, colors=colors)
Map

Map(center=[28.00142, -81.7424], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
fuels = [
    'Coal',
    'Oil',
    'Gas',
    'Hydro',
    'Nuclear',
    'Solar',
    'Waste',
    'Wind',
    'Geothermal',
    'Biomass',
]
colors = [
    '000000',
    '593704',
    'BC80BD',
    '0565A6',
    'E31A1C',
    'FF7F00',
    '6A3D9A',
    '5CA2D1',
    'FDBF6F',
    '229A00',
]

In [ ]:
fc = ee.FeatureCollection("WRI/GPPD/power_plants").filter(
    ee.Filter.inList('fuel1', fuels)
)
styled_fc = geemap.ee_vector_style(fc, column="fuel1", labels=fuels, color=colors)

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.addLayer(styled_fc, {}, 'Power Plants')
Map.add_legend(title="Power Plant Fuel Type", labels=fuels, colors=colors)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

-----------------------------------------------------

## Ejercicio

In [ ]:
# Load a FeatureCollection from a table dataset: 'RESOLVE' ecoregions.
region = Map.user_roi
rectangle = ee.Geometry.Rectangle([-12.29, 35.699, 5.8154, 45.2646])
if region is None:
    region = ee.Geometry.BBox(-12.29, 35.699, 5.8154, 45.2646)
ecoregions = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017').filterBounds(region)
Map.addLayer(ecoregions, {}, 'Ecoregiones')
Map.addLayer(rectangle, {}, 'Rectangle')
Map


Map(bottom=4004.0, center=[23.241346102386135, 355.25390625], controls=(WidgetControl(options=['position', 'tr…

In [ ]:
# Map.user_roi_coords()

[-12.29, 35.699, 5.8154, 45.2646]

In [ ]:
## Creamos los tipos y los colores asociados
types = [
    'Iberian sclerophyllous and semi-deciduous forests',
    'Cantabrian mixed forests',
    'Northwest Iberian montane forests',
    'Southwest Iberian Mediterranean sclerophyllous and mixed forests',
    'Canary Islands dry woodlands and forests',
    'Mediterranean Acacia-Argania dry woodlands and succulent thickets',
    'Pyrenees conifer and mixed forests',
   'Northeast Spain and Southern France Mediterranean forests',
   'Iberian conifer forests',
   'Southeast Iberian shrubs and woodlands'
]

colors = [
    "#008837",
    "#7FC31C",
    "#688CC0",
    "#66C2A5",
    "#0190BF",
    "#13007C",
    "#007C88",
    "#B28653",
    '#CD8966',
    '#C63901'
]

fillColor = [c + "A6" for c in colors]

In [ ]:
fillColor

['#008837A6',
 '#7FC31CA6',
 '#688CC0A6',
 '#66C2A5A6',
 '#0190BFA6',
 '#13007CA6',
 '#007C88A6',
 '#B28653A6',
 '#CD8966A6',
 '#C63901A6']

In [ ]:
fc = ee.FeatureCollection('RESOLVE/ECOREGIONS/2017').filter(
    ee.Filter.inList('ECO_NAME', types)
)
styled_fc = geemap.ee_vector_style(fc, column="ECO_NAME", labels=types, color=colors)

In [ ]:
Map = geemap.Map(center=[40, -1], zoom=4)
Map.addLayer(styled_fc, {}, 'Power Plants')
Map.add_legend(title="Eco regions", labels=types, colors=colors)
Map

Map(center=[40, -1], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

---------------------------------------------------------------------------

In [ ]:
types = ['I', 'U', 'S', 'M', 'C', 'O']
labels = ['Interstate', 'U.S.', 'State recognized', 'Common Name', 'County', 'Other']
colors = ['E31A1C', 'FF7F00', '6A3D9A', '000000', 'FDBF6F', '229A00']
width = [8, 5, 4, 2, 1, 1]

In [ ]:
fc = ee.FeatureCollection('TIGER/2016/Roads')
styled_fc = geemap.ee_vector_style(
    fc, column='rttyp', labels=types, color=colors, width=width
)

In [ ]:
Map = geemap.Map(center=[40.7424, -73.9724], zoom=13)
Map.addLayer(styled_fc, {}, 'Census Roads')
Map.add_legend(title='Route Type', labels=labels, colors=colors)
Map

Map(center=[40.7424, -73.9724], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

## Earth Engine Data Catalog

### Searching for datasets

In [ ]:
dataset_xyz = ee.Image('USGS/SRTMGL1_003')
Map.addLayer(dataset_xyz, {}, "USGS/SRTMGL1_003")

Aqui aprendemos como añadir una **barra de color** para variables continuas!!

In [ ]:
Map = geemap.Map()
dem = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(dem, vis_params, 'SRTM DEM')

#AÑADIMOS BARRA DE COLOR
Map.add_colorbar(vis_params, label="Elevation (m)", layer_name='SRTM DEM', 
                 orientation='vertical', transparent_bg=True, font_size=15)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

El MDT de Francia fue obtenido buscando el catálogo directamente desde de el visor de geemap

In [190]:
Map_france = geemap.Map()

dataset_uee = ee.Image('projects/sat-io/open-datasets/IGN_RGE_Alti_5m')
vis_params = {
    'min': 0,
    'max': 5000,
    'palette': 'terrain',
}
Map_france.addLayer(dataset_uee, vis_params, 'projects/sat-io/open-datasets/IGN_RGE_Alti_5m')
Map_france.setCenter(10, 46, 5)
Map_france

Map(center=[46, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

Jugamos con el GHM v.1.5

In [ ]:

Map_GHM = geemap.Map()

dataset_qkf = ee.Image('projects/sat-io/open-datasets/GHM/ghm_v15_2017_300_60land')
vis_params = {
    'min': 0,
    'max': 1,
    'palette': 'terrain',
}
Map_GHM.addLayer(dataset_qkf, {}, 'projects/sat-io/open-datasets/GHM/ghm_v15_2017_300_60land')
Map_GHM.setCenter(10, 46, 5)
Map_GHM


Map(center=[46, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [184]:
Map_landslide = geemap.Map()
dataset_lands = ee.FeatureCollection('projects/sat-io/open-datasets/events/global_landslide_1970-2019')

Map_landslide.addLayer(dataset_lands, {}, 'projects/sat-io/open-datasets/events/global_landslide_1970-2019')
Map_landslide.setCenter(10, 46, 5)
Map_landslide

Map(center=[46, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

prendemos ahora como crear leyendas prefabricadas / (15.7.1. Built-in legends) --> por ejemplo vamos a cargar el raster de usos del suelo y vamos a utilizar la leyenda predeterminada que viene

In [ ]:
# Primero consutamos las leyendas built-in que hay dentro de GEE -> nos quedamos con la de ESA_Worldcover
from geemap.legends import builtin_legends

for legend in builtin_legends:
    print(legend)

NLCD
ESA_WorldCover
ESRI_LandCover
ESRI_LandCover_TS
Dynamic_World
NWI
MODIS/051/MCD12Q1
MODIS/006/MCD12Q1
GLOBCOVER
JAXA/PALSAR
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL
ALOS_landforms


In [ ]:
Map_LC =geemap.Map()
dataset = ee.ImageCollection('ESA/WorldCover/v200').first()

visualization = {
    'bands' : ['Map'],
}

Map_LC.center_object(dataset)

Map_LC.addLayer(dataset, visualization, 'Landcover')
Map_LC.add_legend(builtin_legend='ESA_WorldCover') ## PARA AÑADIR LA LEYENDA BUILT-IN -> TOMA EL NOMBRE DE LA LISTA
Map_LC

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Ahora se enseña como crear leyendas personalizadas --> visualizar 15.7.2. Custom legends

In [179]:
Map_LC2 = geemap.Map(center=[40, -100], zoom=4)

legend_dict = {
    '11 Open Water': '466b9f',
    '12 Perennial Ice/Snow': 'd1def8',
    '21 Developed, Open Space': 'dec5c5',
    '22 Developed, Low Intensity': 'd99282',
    '23 Developed, Medium Intensity': 'eb0000',
    '24 Developed High Intensity': 'ab0000',
    '31 Barren Land (Rock/Sand/Clay)': 'b3ac9f',
    '41 Deciduous Forest': '68ab5f',
    '42 Evergreen Forest': '1c5f2c',
    '43 Mixed Forest': 'b5c58f',
    '51 Dwarf Scrub': 'af963c',
    '52 Shrub/Scrub': 'ccb879',
    '71 Grassland/Herbaceous': 'dfdfc2',
    '72 Sedge/Herbaceous': 'd1d182',
    '73 Lichens': 'a3cc51',
    '74 Moss': '82ba9e',
    '81 Pasture/Hay': 'dcd939',
    '82 Cultivated Crops': 'ab6c28',
    '90 Woody Wetlands': 'b8d9eb',
    '95 Emergent Herbaceous Wetlands': '6c9fb8',
}

nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = nlcd.select('landcover')

Map_LC2.addLayer(landcover, {}, 'NLCD Land Cover 2019')
Map_LC2.add_legend(title="NLCD Land Cover Classification", legend_dict=legend_dict)
Map_LC2


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

### Using the datasets module

si importas DATA, te puedes ahorrar lo de buscarlo de forma interactiva en el mapa -> util para filtrar con el buscador y completar con autoayuda. Ejm: quiero buscar un producto de la ESA -> pongo ESA y despliego las sugerencias. 

In [180]:
from geemap.datasets import DATA

In [181]:
Map = geemap.Map(center=[40, -100], zoom=4)
dataset = ee.Image(DATA.USGS_GAP_CONUS_2011)
Map.addLayer(dataset, {}, 'GAP CONUS')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [182]:
from geemap.datasets import get_metadata

get_metadata(DATA.USGS_GAP_CONUS_2011)

HTML(value='\n            <html>\n            <body>\n                <h3>USGS GAP CONUS 2011</h3>\n          …

In [188]:
# Al final del video revela uno de los comandos más útiles. Solamente aplicable a feature collection --> geemap.ee_to_df(dataset_lands)


# Al final del video revela uno de los comandos más útiles. Solamente aplicable a feature collection --> **geemap.ee_to_df()**

###  Lo CONVIERTE EN UN DF -> COMO UNA TABLA DE ATRIBUTOS

In [185]:
## Visualizamos el dataset de los deslizamientos utilizando esta función
geemap.ee_to_df(dataset_lands)

,admin_divi,country_co,country_na,created_da,event_date,event_desc,event_id,event_impo,event_time,event_titl,...,notes,photo_link,source_lin,source_nam,storm_name,submitted_,injury_cou,admin_di_1,event_im_1,gazeteer_d
0,,,,11/20/2017 03:17:00 PM,07/03/2017 06:04:00 PM,A couple died after being buried in a landslid...,9955,,22:00,Landslide in Bhojpur,...,,,http://www.myrepublica.com/news/4984/,My Republica,,07/03/2017 06:04:00 PM,NaN,NaN,NaN,NaN
1,,,,11/20/2017 03:17:00 PM,07/05/2017 06:52:00 PM,Transportation has come to a grinding halt alo...,9991,,,Landslide obstructs Pasang Lhamu highway,...,,,http://www.myrepublica.com/news/5434/,My Republica,,07/05/2017 06:52:00 PM,NaN,NaN,NaN,NaN
2,,,,11/20/2017 03:17:00 PM,12/15/2016 02:22:00 PM,Traffic on a major east-west highway from Redd...,10230,,7:00,Rockslide on HWY 299,...,,,http://www.sacbee.com/news/local/transportatio...,The Sacramento Bee,,07/26/2017 01:22:00 PM,0.0,NaN,NaN,NaN
3,,,,12/20/2017 04:36:00 PM,09/21/2016 02:24:00 PM,The Biratnagar-based Eastern Regional Police O...,11184,,,Landslide in Dhankuta,...,,,https://thehimalayantimes.com/nepal/landslides...,The Himalayan Times,,07/11/2017 02:24:00 PM,2.0,NaN,NaN,NaN
4,,,,12/20/2017 04:36:00 PM,07/10/2017 12:55:00 PM,Many vehicles plying along the Rongey/Chongey ...,11183,,,Cars crushed by deadly landslide on Chongey ro...,...,Cannot find location of Maney Khola Rongay,https://static.thenortheasttoday.com/wp-conten...,https://thenortheasttoday.com/cars-crushed-by-...,The Northeast Today,,07/10/2017 12:55:00 PM,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11028,,,,11/20/2017 03:17:00 PM,03/01/2017 06:03:00 PM,The March 1 mudslide that brought layers of se...,10283,,unknown,Big Shannon Run Road Mudslide from Grading Aro...,...,,http://www.observer-reporter.com/storyimage/OR...,http://www.observer-reporter.com/20170308/crew...,Observer-Reporter,,10/06/2017 05:03:00 PM,0.0,NaN,NaN,NaN
11029,England,GB,United Kingdom,11/20/2017 03:17:00 PM,02/09/2016 12:00:00 AM,The Settle-Carlisle line was closed to passeng...,9131,glc,,"Eden Brow, two miles north of Armathwaite, on ...",...,,,http://uk.makemefeed.com/2016/06/13/settlecarl...,Make Me Feed,,07/22/2016 06:31:00 PM,0.0,1575.0,9131.0,8.44909
11030,Rājshāhi,BD,Bangladesh,11/20/2017 03:17:00 PM,04/19/2015 09:00:00 AM,"At a stage, the mud from nearby dam slided on ...",7134,glc,9:00,Ujirpur barrage 3 under Shibchar upazila in Ch...,...,,,http://www.dhakatribune.com/bangladesh/2015/ap...,Dhaka Tribune,,07/10/2015 03:16:00 PM,0.0,142361.0,7134.0,9.97116
11031,Almaty Oblysy,KZ,Kazakhstan,11/20/2017 03:17:00 PM,07/23/2015 02:15:00 AM,"“At around 2.15 am (today, July 23 -ed.) a mor...",7260,glc,2:00,"Kargaly River, Chapaevo Settlement in the Naur...",...,,http://nurotan.kz/sites/default/files/images/2...,http://www.silkroadreporters.com/2015/07/23/mu...,Silk Road Reporters,,10/29/2015 01:51:00 AM,6.0,2000900.0,7260.0,9.83951


## Getting image metadata

In [ ]:
image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_044034_20220503')

In [ ]:
image.bandNames()

In [ ]:
image.select('SR_B1').projection()

In [ ]:
image.select('SR_B1').projection().nominalScale()

In [ ]:
image.propertyNames()

In [ ]:
image.get('CLOUD_COVER')

In [ ]:
image.get('DATE_ACQUIRED')

In [ ]:
image.get('system:time_start')

In [ ]:
date = ee.Date(image.get('system:time_start'))
date.format('YYYY-MM-dd')

In [ ]:
image.toDictionary()

In [ ]:
props = geemap.image_props(image)
props

## Calculating descriptive statistics

In [ ]:
image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_044034_20220503')
geemap.image_min_value(image)

In [ ]:
geemap.image_max_value(image)

In [ ]:
geemap.image_mean_value(image)

In [ ]:
geemap.image_stats(image)

## Using the inspector tool

In [ ]:
Map = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(
    ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(dem, vis_params, 'SRTM DEM')
Map.addLayer(
    landsat7,
    {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2.0},
    'Landsat 7',
)
Map.addLayer(states, {}, "US States")
Map

## Converting JavaScript to Python

### Interactive conversion

In [ ]:
Map = geemap.Map()
Map

In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
vizParams = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

# Center the map and display the image.
Map.setCenter(-122.1899, 37.5010, 10)
# San Francisco Bay
Map.addLayer(image, vizParams, 'False color composite')

### Batch conversion

In [ ]:
snippet = """
// Load an image.
var image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318');

// Create an NDWI image, define visualization parameters and display.
var ndwi = image.normalizedDifference(['B3', 'B5']);
var ndwiViz = {min: 0.5, max: 1, palette: ['00FFFF', '0000FF']};
Map.addLayer(ndwi, ndwiViz, 'NDWI');
Map.centerObject(image)
"""

geemap.js_snippet_to_py(snippet, add_new_cell=True, import_ee=False)

In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.addLayer(ndwi, ndwiViz, 'NDWI')
Map.centerObject(image)
Map

In [ ]:
import os
from geemap.conversion import *

out_dir = os.getcwd()
js_dir = get_js_examples(out_dir)
js_to_python_dir(in_dir=js_dir, out_dir=out_dir, use_qgis=False)
py_to_ipynb_dir(js_dir)

## Calling JavaScript functions from Python

In [ ]:
# %pip install oeel

In [ ]:
import oeel

In [ ]:
oeel = geemap.requireJS()

In [ ]:
ic = ee.ImageCollection("COPERNICUS/S2_SR")
icSize = (
    oeel.Algorithms.Sentinel2.cloudfree(maxCloud=20, S2Collection=ic)
    .filterDate('2020-01-01', '2020-01-02')
    .size()
)
print('Cloud free imagery: ', icSize.getInfo())

In [ ]:
url = 'https://tinyurl.com/27xy4oh9'
lib = geemap.requireJS(lib_path=url)

In [ ]:
lib.availability

In [ ]:
grid = lib.generateGrid(-180, -70, 180, 70, 10, 10, 0, 0)
grid.first()

In [ ]:
Map = geemap.Map()
style = {'fillColor': '00000000'}
Map.addLayer(grid.style(**style), {}, 'Grid')
Map

In [ ]:
Map = geemap.Map()
lib = geemap.requireJS(lib_path='grid.js', Map=Map)

In [ ]:
grid = lib.generateGrid(-180, -70, 180, 70, 10, 10, 0, 0)
grid.first()

In [ ]:
lib.grid_test()
Map

In [ ]:
lib = geemap.requireJS('users/gena/packages:grid')

In [ ]:
grid = lib.generateGrid(-180, -70, 180, 70, 10, 10, 0, 0)

In [ ]:
Map = geemap.Map()
style = {'fillColor': '00000000'}
Map.addLayer(grid.style(**style), {}, 'Grid')
Map

## Summary